<a href="https://colab.research.google.com/github/LKXoro/BEDU-Rstudio-postworks/blob/main/M3_Postworks5%2C6/Postwork_05_06_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Postworks 05 y 06: Limpieza de datos y agregaciones / Automatización y APIs

En esencia el Postwork 05 busca que limpiemos nuestro Dataset de valores nulos y probemos obtener información mediante agreaciones. Mientras que el Postwork 06 invita a automatizar el proceso de peticiones a una API. En esta entrega cambiaremos un poco el orden, primero realizaremos la automatización de la petición para obtener todos los datos y después los limpiaremos.

<p align="center">
<img src="https://empresas.blogthinkbig.com/wp-content/uploads/2019/03/janitor.png" height ="200" width = "200" alt=lego>
</p>

En postworks anteriores mencionamos que una petición a nuestra API de confianza, PandaScore, puede regresar como máximo 100 registros por petición y que las peticiones responden a parámetros de la URL. Creemos que la mejor forma de automatizar este proceso es con una función que nos devuelva la información que queremos, del juego que queremos de la API:


In [27]:
import requests #Importamos los paquete necesarios
import pandas as pd 
import math as mt
 
def PandaScore_get(juego,informacion): #Definimos nuestra función
    if isinstance(juego, str) and isinstance(informacion, str): #Nos aseguramos que los argumentos sean de tipo str
        registros = 100 #Establecemos el número de registros por página
        token = 'V3U1fnnWozJi3bbhZfrxdiAmN4-XWnX_4g6i_6iauwwh7w-V8Ns' #Token de acceso
        url = f"https://api.pandascore.co/{juego}/{informacion}?page[size]={registros}&token={token}" #URL con los parámetros deseados
        response = requests.get(url) #Realizamos la petición
        if response.status_code == 200: #Si todo salió bien continuamos
            Iteraciones = mt.ceil(float(response.headers['X-Total'])/registros) #Calculamos el número de páginas en base a todos los registros que hay en la API
            #Al dividir el número total de registros entre los registros solicitados obtenemos un aproximado, para asegurar que tomaremos todos, redondeamos al entero más alto
            for i in range(1,Iteraciones+1): #Ciclo for para obtener todos los registros de cada página
                url = f"https://api.pandascore.co/{juego}/{informacion}?page[size]={registros}&page[number]={i}&token=V3U1fnnWozJi3bbhZfrxdiAmN4-XWnX_4g6i_6iauwwh7w-V8Ns"
                #URL el parámetro de página deseada
                response = requests.get(url) #Realizamos la petición correspondiente
                data = response.json() #Transformamos la respuesta 
                if i == 1: #En la primer iteración se crea un Dataframe
                    df = pd.DataFrame.from_dict(data)
                else: #En el resto de iteraciones se van añadiendo los nuevos registros
                    df = df.append(pd.DataFrame.from_dict(data),ignore_index=True) 
            return df # Finalmente se devuelve el Dataframe con todos los registros
        else: #Si la petición no se ejecutó correctamente mandamos un mensaje con el error devuelto
            return 'Error de petición: '+str(response.status_code)  
    else: #Si los argumentos no son de tipo str regresamos un mensaje
        return 'Por favor coloque un argumento válido en formato string.'
  

Veamos a la función en acción:

In [31]:
PandaScore_get('lol',1)

'Por favor coloque un argumento válido en formato string.'

In [32]:
PandaScore_get('lol','matchs')

'Error de petición: 404'

Si los argumentos son correctos se nos devolverá el Dataframe correspondiente, el tiempo de ejecución varia dependiendo de la información solicitada, en el caso de los enfrentamientos tenemos 18694 registros, por lo que la función tomará un par de minutos al solo poder realizar 100 peticiones, en general consumiriamos 187 de nuestros 1000 intentos por hora:

In [ ]:
df_matches = PandaScore_get('lol','matches')

Observemos el Dataframe que se nos devuelve:

In [42]:
df_matches

,forfeit,number_of_games,name,slug,videogame,begin_at,tournament,streams,results,winner_id,modified_at,league,official_stream_url,draw,winner,games,league_id,live_embed_url,videogame_version,status,game_advantage,serie_id,tournament_id,scheduled_at,id,serie,match_type,end_at,original_scheduled_at,detailed_stats,rescheduled,opponents,live
0,False,5,Round 3: TBD vs TBD,2021-03-11-c3b8c131-fac1-4b71-b339-f1d0387914a6,"{'id': 1, 'name': 'LoL', 'slug': 'league-of-le...",2021-03-11T17:00:00Z,"{'begin_at': '2021-03-09T17:00:00Z', 'end_at':...","{'english': {'embed_url': None, 'raw_url': Non...",[],NaN,2021-03-07T20:20:32Z,"{'id': 4302, 'image_url': 'https://cdn.pandasc...",None,False,None,"[{'begin_at': None, 'complete': False, 'detail...",4302,None,None,not_started,None,3254,5649,2021-03-11T17:00:00Z,586007,"{'begin_at': '2021-01-19T17:00:00Z', 'descript...",best_of,None,2021-03-11T17:00:00Z,True,False,[],"{'opens_at': None, 'supported': False, 'url': ..."
1,False,3,Round 2 Match 2: MOUZ vs BIG,mousesports-vs-berlin-international-gaming-202...,"{'id': 1, 'name': 'LoL', 'slug': 'league-of-le...",2021-03-10T20:00:00Z,"{'begin_at': '2021-03-09T17:00:00Z', 'end_at':...","{'english': {'embed_url': None, 'raw_url': Non...","[{'score': 0, 'team_id': 16}, {'score': 0, 'te...",NaN,2021-03-07T20:19:53Z,"{'id': 4302, 'image_url': 'https://cdn.pandasc...",None,False,None,"[{'begin_at': None, 'complete': False, 'detail...",4302,None,None,not_started,None,3254,5649,2021-03-10T20:00:00Z,586006,"{'begin_at': '2021-01-19T17:00:00Z', 'descript...",best_of,None,2021-03-10T20:00:00Z,True,False,"[{'opponent': {'acronym': 'MOUZ', 'id': 16, 'i...","{'opens_at': None, 'supported': False, 'url': ..."
2,False,3,Round 2 Match 1: TBD vs TBD,2021-03-10-7412caf0-3f45-40e1-b763-80d005a04162,"{'id': 1, 'name': 'LoL', 'slug': 'league-of-le...",2021-03-10T17:00:00Z,"{'begin_at': '2021-03-09T17:00:00Z', 'end_at':...","{'english': {'embed_url': None, 'raw_url': Non...",[],NaN,2021-03-07T20:19:33Z,"{'id': 4302, 'image_url': 'https://cdn.pandasc...",None,False,None,"[{'begin_at': None, 'complete': False, 'detail...",4302,None,None,not_started,None,3254,5649,2021-03-10T17:00:00Z,586005,"{'begin_at': '2021-01-19T17:00:00Z', 'descript...",best_of,None,2021-03-10T17:00:00Z,True,False,[],"{'opens_at': None, 'supported': False, 'url': ..."
3,False,3,Round 1 Match 2: SKP vs S04.E,sk-gaming-prime-vs-fc-schalke-04-evolution-202...,"{'id': 1, 'name': 'LoL', 'slug': 'league-of-le...",2021-03-09T20:00:00Z,"{'begin_at': '2021-03-09T17:00:00Z', 'end_at':...","{'english': {'embed_url': None, 'raw_url': Non...","[{'score': 0, 'team_id': 247}, {'score': 0, 't...",NaN,2021-03-07T20:18:12Z,"{'id': 4302, 'image_url': 'https://cdn.pandasc...",None,False,None,"[{'begin_at': None, 'complete': False, 'detail...",4302,None,None,not_started,None,3254,5649,2021-03-09T20:00:00Z,586004,"{'begin_at': '2021-01-19T17:00:00Z', 'descript...",best_of,None,2021-03-09T20:00:00Z,True,False,"[{'opponent': {'acronym': 'SKP', 'id': 247, 'i...","{'opens_at': None, 'supported': False, 'url': ..."
4,False,3,Round 1 Match 1: GL vs USE,team-gamerlegion-vs-unicorns-of-love-sexy-edit...,"{'id': 1, 'name': 'LoL', 'slug': 'league-of-le...",2021-03-09T17:00:00Z,"{'begin_at': '2021-03-09T17:00:00Z', 'end_at':...","{'english': {'embed_url': None, 'raw_url': Non...","[{'score': 0, 'team_id': 126830}, {'score': 0,...",NaN,2021-03-07T20:17:44Z,"{'id': 4302, 'image_url': 'https://cdn.pandasc...",None,False,None,"[{'begin_at': None, 'complete': False, 'detail...",4302,None,None,not_started,None,3254,5649,2021-03-09T17:00:00Z,586003,"{'begin_at': '2021-01-19T17:00:00Z', 'descript...",best_of,None,2021-03-09T17:00:00Z,True,False,"[{'opponent': {'acronym': 'GL', 'id': 126830, ...","{'opens_at': None, 'supported': False, 'url': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18689,False,1,DIG vs C9,dignitas-vs-cloud9-2015-05-31,"{'id': 1, 

Veamos las columnas y tipos de datos:

In [34]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18694 entries, 0 to 18693
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   forfeit                18694 non-null  bool   
 1   number_of_games        18694 non-null  int64  
 2   name                   18694 non-null  object 
 3   slug                   18694 non-null  object 
 4   videogame              18694 non-null  object 
 5   begin_at               18662 non-null  object 
 6   tournament             18694 non-null  object 
 7   streams                18694 non-null  object 
 8   results                18694 non-null  object 
 9   winner_id              17758 non-null  float64
 10  modified_at            18694 non-null  object 
 11  league                 18694 non-null  object 
 12  official_stream_url    10896 non-null  object 
 13  draw                   18694 non-null  bool   
 14  winner                 17758 non-null  object 
 15  ga

Como podemos ver tenemos 33 columnas, un número considerable, observemos que contienen algunas y si nos son de utilidad. Empecemos por los campos que contienen listas:

In [44]:
df_matches['videogame'][0] 

{'id': 1, 'name': 'LoL', 'slug': 'league-of-legends'}

El campo _videogame_ puede que haya sido de utilidad para la API al momento de regresarnos una respuesta, sin embargo, es  redundante para nosotros.

In [145]:
df_matches['slug']

0          2021-03-11-c3b8c131-fac1-4b71-b339-f1d0387914a6
1        mousesports-vs-berlin-international-gaming-202...
2          2021-03-10-7412caf0-3f45-40e1-b763-80d005a04162
3        sk-gaming-prime-vs-fc-schalke-04-evolution-202...
4        team-gamerlegion-vs-unicorns-of-love-sexy-edit...
                               ...                        
18689                        dignitas-vs-cloud9-2015-05-31
18690                     team-8-vs-team-liquid-2015-05-31
18691                          gravity-vs-enemy-2015-05-31
18692                         team-8-vs-impulse-2015-05-31
18693             team-liquid-vs-dragon-knights-2015-05-30
Name: slug, Length: 18694, dtype: object

Una combinación del campo nombre y la fecha programada. Además de que en algunos casos tenemos valores raros, consideramos que podemos retirar esta columna.

In [48]:
df_matches['streams'][0]

{'english': {'embed_url': None, 'raw_url': None},
 'official': {'embed_url': None, 'raw_url': None},
 'russian': {'embed_url': None, 'raw_url': None}}

In [47]:
df_matches['streams'][300]

{'english': {'embed_url': None, 'raw_url': None},
 'official': {'embed_url': 'https://player.twitch.tv/?channel=riotgamesru',
  'raw_url': 'https://www.twitch.tv/riotgamesru'},
 'russian': {'embed_url': 'https://player.twitch.tv/?channel=riotgamesru',
  'raw_url': 'https://www.twitch.tv/riotgamesru'}}

In [57]:
df_matches.loc[300,'official_stream_url']

'https://www.twitch.tv/riotgamesru'

In [162]:
df_matches['live_embed_url'][300]

'https://player.twitch.tv/?channel=riotgamesru'

In [163]:
df_matches['official_stream_url'][300]

'https://www.twitch.tv/riotgamesru'

El campo _streams_ nos regresa las direcciones URL de las transmisiones en vivo en diferentes idiomas, mientras que el campo _live embed url_ nos da el enlace en el que se transmitió en vivo y _official stream url_ nos regresa únicamente la URL del canal oficial que realizo el streaming. Creemos es mejor conservar este último campo con la dirección oficial a un campo con una lista  más difícil de manejar y otro con URLs que ya no son accesibles.

In [60]:
df_matches['tournament'][0]

{'begin_at': '2021-03-09T17:00:00Z',
 'end_at': None,
 'id': 5649,
 'league_id': 4302,
 'live_supported': False,
 'modified_at': '2021-03-08T14:50:51Z',
 'name': 'Playoffs',
 'prizepool': None,
 'serie_id': 3254,
 'slug': 'league-of-legends-prime-league-pro-division-spring-2021-playoffs',
 'winner_id': None,
 'winner_type': 'Team'}

El campo _tournament_ nos devuelve el torneo correspondiente. Sin embargo, contamos con el _tournament id_, y podemos realizar peticiones de torneos si lo deseamos, por lo que de ser necesario podriamos realizar un **join** y observar a que torneo corresponde cada enfrentamiento. 

In [65]:
df_matches['modified_at'].head() 

0    2021-03-07T20:20:32Z
1    2021-03-07T20:19:53Z
2    2021-03-07T20:19:33Z
3    2021-03-07T20:18:12Z
4    2021-03-07T20:17:44Z
Name: modified_at, dtype: object

La última fecha de modificación respecto a la API no nos resulta útil del todo a nosotros.

In [66]:
df_matches['league'][0]

{'id': 4302,
 'image_url': 'https://cdn.pandascore.co/images/league/image/4302/220px-Prime_Leaguelogo_square.png',
 'modified_at': '2020-01-20T16:20:40Z',
 'name': 'Prime League Pro Division',
 'slug': 'league-of-legends-prime-league-pro-division',
 'url': 'https://www.primeleague.gg/de/start'}

In [98]:
df_matches['serie'][0]

{'begin_at': '2021-01-19T17:00:00Z',
 'description': None,
 'end_at': None,
 'full_name': 'Spring 2021',
 'id': 3254,
 'league_id': 4302,
 'modified_at': '2021-01-12T08:59:18Z',
 'name': '',
 'season': 'Spring',
 'slug': 'league-of-legends-prime-league-pro-division-spring-2021',
 'tier': 'c',
 'winner_id': None,
 'winner_type': None,
 'year': 2021}

Mismo caso que con los torneos, si conservamos el _league id_ y _serie id_ no deberiamos tener problemas.

In [188]:
df_matches['id']

0        586007
1        586006
2        586005
3        586004
4        586003
          ...  
18689         7
18690         6
18691         5
18692         4
18693         2
Name: id, Length: 18694, dtype: int64

Id asignado por la API a cada enfrentamiento, con el observamos que los enfrentamientos en los primeros registros son los más recientes y los últmos son los más antiguos. Para evitar confusiones y por el tratamiento que le daremos a los datos será mejor usar nuestra propia indexación. 

In [70]:
df_matches['games'][300]

[{'begin_at': None,
  'complete': False,
  'detailed_stats': True,
  'end_at': None,
  'finished': False,
  'forfeit': False,
  'id': 222685,
  'length': None,
  'match_id': 583283,
  'position': 1,
  'status': 'not_started',
  'video_url': None,
  'winner': {'id': None, 'type': None},
  'winner_type': None}]

El campo _games_ debería entregar información de cada partida en el enfrentamiento, pero en general esos datos podemos encontrarlos en otros campos que ya tenemos.

In [96]:
df_matches['game_advantage'].unique()

array([None, nan, 2862.0, 318.0, 126059.0, 329.0, 74.0, 1566.0, 249.0,
       205.0, 411.0, 65.0, 267.0], dtype=object)

El campo game_advantage es poco claro, en la documentación no se específica el significado de estos valores.

In [99]:
df_matches['original_scheduled_at']

0        2021-03-11T17:00:00Z
1        2021-03-10T20:00:00Z
2        2021-03-10T17:00:00Z
3        2021-03-09T20:00:00Z
4        2021-03-09T17:00:00Z
                 ...         
18689                    None
18690                    None
18691                    None
18692                    None
18693                    None
Name: original_scheduled_at, Length: 18694, dtype: object

In [97]:
df_matches['scheduled_at']

0        2021-03-11T17:00:00Z
1        2021-03-10T20:00:00Z
2        2021-03-10T17:00:00Z
3        2021-03-09T20:00:00Z
4        2021-03-09T17:00:00Z
                 ...         
18689    2015-05-31T20:00:00Z
18690    2015-05-31T18:00:00Z
18691    2015-05-30T23:00:00Z
18692    2015-05-30T22:00:00Z
18693    2015-05-30T20:00:00Z
Name: scheduled_at, Length: 18694, dtype: object

Estos campos pueden indicar modificaciones en las fechas de los enfrentamientos, pero el campo _rescheduled_ nos brinda la misma información de forma más concisa.

In [100]:
df_matches['match_type'].unique()

array(['best_of'], dtype=object)

En general todos los enfrentamientos son del tipo "_el mejor de_", así que este campo también es redundante.

In [117]:
df_matches['detailed_stats']

0        True
1        True
2        True
3        True
4        True
         ... 
18689    True
18690    True
18691    True
18692    True
18693    True
Name: detailed_stats, Length: 18694, dtype: bool

Un campo que indica si el campo _games_ contiene detalles, pero ya que no vamos a utilizar _games_ podemos omitirlo.

In [138]:
df_matches['videogame_version']

0                                       None
1                                       None
2                                       None
3                                       None
4                                       None
                        ...                 
18689    {'current': False, 'name': '5.8.1'}
18690    {'current': False, 'name': '5.8.1'}
18691    {'current': False, 'name': '5.8.1'}
18692    {'current': False, 'name': '5.8.1'}
18693    {'current': False, 'name': '5.9.1'}
Name: videogame_version, Length: 18694, dtype: object

Un campo que podría ser de interés si analizamos las composiciones de _personajes_ de cada equipo respecto a la versión del juego, lamentablemente no contamos con esa información.

In [111]:
df_matches['live'][0]

{'opens_at': None, 'supported': False, 'url': None}

Finalmente el campo _live_ nos brinda la url del stream si el enfrentamiento está en vivo en ese momento, en general si buscaramos un soporte en vivo sería un campo genial, pero como deseamos trabajar con resultados de enfrentamientos que ya han terminado podemos trabajar sin este campo.

Ahora eliminamos los campos que deseamos retirar:

In [189]:
campos_no_deseados = ['videogame', 'slug', 'tournament', 'league', 'serie', 'streams', 'modified_at', 'games',
                      'game_advantage', 'scheduled_at', 'original_scheduled_at', 'detailed_stats','live',
                      'match_type', 'videogame_version', 'live_embed_url', 'id']

df_matches_temp = df_matches.drop(columns=campos_no_deseados)
df_matches_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18694 entries, 0 to 18693
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   forfeit              18694 non-null  bool   
 1   number_of_games      18694 non-null  int64  
 2   name                 18694 non-null  object 
 3   begin_at             18662 non-null  object 
 4   results              18694 non-null  object 
 5   winner_id            17758 non-null  float64
 6   official_stream_url  10896 non-null  object 
 7   draw                 18694 non-null  bool   
 8   winner               17758 non-null  object 
 9   league_id            18694 non-null  int64  
 10  status               18694 non-null  object 
 11  serie_id             18694 non-null  int64  
 12  tournament_id        18694 non-null  int64  
 13  end_at               18123 non-null  object 
 14  rescheduled          18694 non-null  bool   
 15  opponents            18694 non-null 

In [190]:
df_matches_temp.head()

,forfeit,number_of_games,name,begin_at,results,winner_id,official_stream_url,draw,winner,league_id,status,serie_id,tournament_id,end_at,rescheduled,opponents
0,False,5,Round 3: TBD vs TBD,2021-03-11T17:00:00Z,[],NaN,None,False,None,4302,not_started,3254,5649,None,False,[]
1,False,3,Round 2 Match 2: MOUZ vs BIG,2021-03-10T20:00:00Z,"[{'score': 0, 'team_id': 16}, {'score': 0, 'te...",NaN,None,False,None,4302,not_started,3254,5649,None,False,"[{'opponent': {'acronym': 'MOUZ', 'id': 16, 'i..."
2,False,3,Round 2 Match 1: TBD vs TBD,2021-03-10T17:00:00Z,[],NaN,None,False,None,4302,not_started,3254,5649,None,False,[]
3,False,3,Round 1 Match 2: SKP vs S04.E,2021-03-09T20:00:00Z,"[{'score': 0, 'team_id': 247}, {'score': 0, 't...",NaN,None,False,None,4302,not_started,3254,5649,None,False,"[{'opponent': {'acronym': 'SKP', 'id': 247, 'i..."
4,False,3,Round 1 Match 1: GL vs USE,2021-03-09T17:00:00Z,"[{'score': 0, 'team_id': 126830}, {'score': 0,...",NaN,None,False,None,4302,not_started,3254,5649,None,False,"[{'opponent': {'acronym': 'GL', 'id': 126830, ..."


¡Genial¡, pudimos reducir el número de campos de 33 a 16, ahora veamos de que forma podemos tratar con los valores _None_. Si inspeccionamos por columna:



In [191]:
df_matches_temp.isna().sum(axis=0)

forfeit                   0
number_of_games           0
name                      0
begin_at                 32
results                   0
winner_id               936
official_stream_url    7798
draw                      0
winner                  936
league_id                 0
status                    0
serie_id                  0
tournament_id             0
end_at                  571
rescheduled               0
opponents                 0
dtype: int64

Observamos que la mayor cantidad de valores nulos se da en _official stream url_ esto no es del todo un problema ya que no necesariamente todos los partidos se transmitieron en directo. Entonces nos encontramos con valores nulos en las fechas de inicio y terminó, seguramente por partidos que todavía no se han llevado a cabo, se están llevando a cabo en estos momentos o se cancelaron, por ende también el valor del campo _winner_ tiene un valor también nulo.

Aprovechemos el campo status para limpiar nuestro dataframe:



In [192]:
df_matches_temp['status'].unique()

array(['not_started', 'finished', 'running', 'canceled'], dtype=object)

Obtengamos solo aquellos registros donde el estado del enfrentamiento sea "_finished_":

In [193]:
df_matches_finished = df_matches_temp[ df_matches_temp['status'] == 'finished' ]
df_matches_finished.isna().sum(axis=0)

forfeit                   0
number_of_games           0
name                      0
begin_at                  1
results                   0
winner_id               398
official_stream_url    7772
draw                      0
winner                  398
league_id                 0
status                    0
serie_id                  0
tournament_id             0
end_at                   19
rescheduled               0
opponents                 0
dtype: int64

Aún conservamos un par de valores _None_ en nuestras fechas, esto puede ser a causa de errores en la recopilación de los datos. Deshagámonos de ellos:

In [194]:
df_matches_finished_clean = df_matches_finished.dropna(subset=['begin_at', 'end_at'])
df_matches_finished_clean.isna().sum(axis=0)

forfeit                   0
number_of_games           0
name                      0
begin_at                  0
results                   0
winner_id               398
official_stream_url    7755
draw                      0
winner                  398
league_id                 0
status                    0
serie_id                  0
tournament_id             0
end_at                    0
rescheduled               0
opponents                 0
dtype: int64

!Enhorabuena¡ tenemos un dataframe solo con las partidas que han sido finalizadas y además tienen una fecha de fin e inicio establecidas. ¿Pero qué pasa con esos 398 valores _None_ en el campo _winner id_ y _winner_? ¿Y los 7735 de _official stream url_? Pues tienen una explicación muy sencilla en realidad. Estos datos quieren decir que 398 enfrentamiento han terminado en empate y 7735 enfrentamientos no se transmitieron en línea.

Para dejar nuestro dataframe un poco más presentable vamos a ordenar los campos de formas más amigable:

In [204]:
#No incluiremos al campo status por que ya ha cumplido su propósito
df_matches_ordenado = df_matches_finished_clean[['name','opponents','results','forfeit','draw',
                                                      'number_of_games','winner','winner_id','begin_at','end_at',
                                                      'rescheduled','official_stream_url','league_id', 'serie_id', 'tournament_id']]

Observemos al nuevo Dataframe:

In [202]:
df_matches_ordenado.head()

,name,opponents,results,forfeit,draw,number_of_games,winner,winner_id,begin_at,end_at,rescheduled,official_stream_url,league_id,serie_id,tournament_id
15,ALF vs BJD,"[{'opponent': {'acronym': 'ALF', 'id': 124408,...","[{'score': 0, 'team_id': 124408}, {'score': 1,...",False,False,1,"{'acronym': 'BJD', 'id': 126861, 'image_url': ...",126861.0,2021-03-06T09:54:42Z,2021-03-06T10:28:54Z,False,https://www.twitch.tv/lolpacific,4288,3337,5483
37,Winners bracket round 1 match 2: ESB vs SIN,"[{'opponent': {'acronym': 'ESB', 'id': 3344, '...","[{'score': 2, 'team_id': 3344}, {'score': 3, '...",False,False,5,"{'acronym': 'SIN', 'id': 127586, 'image_url': ...",127586.0,2021-03-07T17:53:43Z,2021-03-07T22:22:51Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602
38,Winners bracket round 1 match 1: IG vs VIK,"[{'opponent': {'acronym': 'IG', 'id': 127585, ...","[{'score': 3, 'team_id': 127585}, {'score': 1,...",False,False,5,"{'acronym': 'IG', 'id': 127585, 'image_url': '...",127585.0,2021-03-07T13:30:41Z,2021-03-07T17:08:57Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602
39,Losers bracket round 1 match 1: DTG vs ECT,"[{'opponent': {'acronym': 'DTG', 'id': 128442,...","[{'score': 3, 'team_id': 128442}, {'score': 1,...",False,False,5,"{'acronym': 'DTG', 'id': 128442, 'image_url': ...",128442.0,2021-03-04T17:29:15Z,2021-03-04T21:22:00Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602
40,Tiebreaker: ESB vs SIN,"[{'opponent': {'acronym': 'ESB', 'id': 3344, '...","[{'score': 1, 'team_id': 3344}, {'score': 0, '...",False,False,1,"{'acronym': 'ESB', 'id': 3344, 'image_url': 'h...",3344.0,2021-02-28T23:08:04Z,2021-02-28T23:57:11Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5436


Reiniciemos el índice y cambiemos algunos nombre de los campos para obtener un Dataframe más limpio:

In [218]:
df_matches_ordenado = df_matches_ordenado.reset_index(drop=True) 
nombres_nuevos = {
    'name' : 'match_name',
    'begin_at' : 'begin_date',
    'end_at' : 'end_date',
    'official_stream_url' : 'official_stream_channel_url'
}

df_matches_ordenado.rename(columns=nombres_nuevos, inplace=True)
df_matches_ordenado.head()

,match_name,opponents,results,forfeit,draw,number_of_games,winner,winner_id,begin_date,end_date,rescheduled,official_stream_channel_url,league_id,serie_id,tournament_id
0,ALF vs BJD,"[{'opponent': {'acronym': 'ALF', 'id': 124408,...","[{'score': 0, 'team_id': 124408}, {'score': 1,...",False,False,1,"{'acronym': 'BJD', 'id': 126861, 'image_url': ...",126861.0,2021-03-06T09:54:42Z,2021-03-06T10:28:54Z,False,https://www.twitch.tv/lolpacific,4288,3337,5483
1,Winners bracket round 1 match 2: ESB vs SIN,"[{'opponent': {'acronym': 'ESB', 'id': 3344, '...","[{'score': 2, 'team_id': 3344}, {'score': 3, '...",False,False,5,"{'acronym': 'SIN', 'id': 127586, 'image_url': ...",127586.0,2021-03-07T17:53:43Z,2021-03-07T22:22:51Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602
2,Winners bracket round 1 match 1: IG vs VIK,"[{'opponent': {'acronym': 'IG', 'id': 127585, ...","[{'score': 3, 'team_id': 127585}, {'score': 1,...",False,False,5,"{'acronym': 'IG', 'id': 127585, 'image_url': '...",127585.0,2021-03-07T13:30:41Z,2021-03-07T17:08:57Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602
3,Losers bracket round 1 match 1: DTG vs ECT,"[{'opponent': {'acronym': 'DTG', 'id': 128442,...","[{'score': 3, 'team_id': 128442}, {'score': 1,...",False,False,5,"{'acronym': 'DTG', 'id': 128442, 'image_url': ...",128442.0,2021-03-04T17:29:15Z,2021-03-04T21:22:00Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602
4,Tiebreaker: ESB vs SIN,"[{'opponent': {'acronym': 'ESB', 'id': 3344, '...","[{'score': 1, 'team_id': 3344}, {'score': 0, '...",False,False,1,"{'acronym': 'ESB', 'id': 3344, 'image_url': 'h...",3344.0,2021-02-28T23:08:04Z,2021-02-28T23:57:11Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5436


!Bien¡ Tenemos un Dataframe más limpio y ordenado, con ayuda de algunos métodos de agregación, podemos aprovechar los valores _none_ que hemos dejado, por ejemplo, para conocer la proporción de enfrentamientos transmitidos: 

In [223]:
sin_stream = df_matches_ordenado['official_stream_channel_url'].isna().sum()
total = len(df_matches_ordenado['official_stream_channel_url'])

proporción = (sin_stream/total)*100
proporción 

42.83583738400353

Ahora sabemos que aproximadamente el 42.83% de los enfrentamientos de lol son transmitidos en línea. Finalmente guardemos nuestro Dataframe en un archivo csv:

In [220]:
df_matches_ordenado.to_csv('lol_matches.csv')